In [1]:
### forest-disturbance-stack-v3

### This script creates a function to create a custom disturbance stack. It accepts
### a template raster path (to determine resolution and extent), a list of
### individual raster file paths (e.g., biotic, wildfire, drought stacks), and a 
### forest mask path (binary mask).
### Returns and saves a final GeoTIFF (.tif).


# If working in cyverse, set working directory and project root
setwd("/home/jovyan/data-store/forest-disturbance-stack-v3")
here::i_am("README.md")   # or any file guaranteed to exist in the project



# Install and load required packages
packages <- c("here", "terra", "fs", "tidyverse", "progressr")
installed <- packages %in% installed.packages()[, "Package"]
if (any(!installed)) {
  install.packages(packages[!installed])
}

library(here)
library(terra)
library(fs)
library(tidyverse)
library(progressr)

# Set cyverse memory max to avoid crashing
terraOptions(memmax=240000)

# Enable progress bars globally
handlers("txtprogressbar")

here() starts at /home/jovyan/data-store/forest-disturbance-stack-v3

terra 1.7.55

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::extract() masks terra::extract()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# ============================================================
#  0. Setup
# ============================================================

# Paths
ref_raster_path <- here("data/derived/wildfire_id.tif")
raster_paths <- c(
  here("data/derived/wildfire_id.tif"),
  here("data/derived/biotic_gridded_1km_all_years_severity.tif"),
  here("data/derived/pdsi_annual.tif"),
  here("data/derived/hd_fingerprint.tif")
)
template_path <- here("data/derived/template_singleband_30m.tif")
forest_mask_path <- here("data/derived/relaxed_forest_mask_2000_2020.tif")
output_stack_path <- here("data/derived/disturbance_stack_2000_2020.tif")

target_years <- 2000:2020
resolution <- 30  # in meters
n_threads <- 8

tmp_dirs <- list(
  template = here("data/derived/tmp_template"),
  subset  = here("data/derived/tmp_subset"),
  resample = here("data/derived/tmp_resample"),
  vrt = here("data/derived/tmp_vrt")
)
lapply(tmp_dirs, dir_create)

ERROR: [1m[33mError[39m in `map()`:[22m
[33m![39m `.x` must be a vector, not a function.


In [3]:
# ============================================================
#  1. Create template raster
# ============================================================

if (!file.exists(template_path)) {
  ref <- rast(ref_raster_path)
  template <- rast(extent = ext(ref), resolution = resolution, crs = crs(ref))
  values(template) <- NA
  
  writeRaster(template, template_path, overwrite=TRUE,
              datatype="FLT4S", gdal=c("COMPRESS=DEFLATE", "TILED=YES"))
  message("✅ Single-band template created: ", template_path)
} else {
  message("✅ Template already exists: ", template_path)
}

✅ Single-band template created: /home/jovyan/data-store/forest-disturbance-stack-v3/data/derived/template_singleband_30m.tif



In [ ]:
# ============================================================
# 2. Subset rasters to 2000–2020 to save memory
# ============================================================

subset_raster_years <- function(raster_path, target_years, out_dir) {
  r <- rast(raster_path)
  layer_years <- as.numeric(sub(".*_(\\d{4})$", "\\1", names(r)))
  keep <- which(layer_years %in% target_years)
  r_subset <- r[[keep]]
  
  out_file <- file.path(out_dir, paste0(tools::file_path_sans_ext(basename(raster_path)), "_subset.tif"))
  writeRaster(r_subset, out_file, overwrite = TRUE,
              datatype = "FLT4S",
              gdal = c("COMPRESS=DEFLATE", "TILED=YES"))
  
  message(sprintf("Raster: %s | Bands kept: %d", basename(raster_path), length(keep)))
  out_file
}

# Run
with_progress({
  p <- progressor(along = raster_paths)
  subset_files <- map(raster_paths, ~{
    p(sprintf("Subsetting: %s", basename(.x)))
    subset_raster_years(.x, target_years, tmp_dirs$subset)
  })
})

In [ ]:
# ============================================================
#  3. Resample Rasters to Template Extent/CRS
# ============================================================

resample_to_template <- function(template_path, raster_paths, out_dir, n_threads = 8) {
  template <- rast(template_path)
  out_files <- c()
  
  with_progress({
    p <- progressor(along = raster_paths)
    for (rp in raster_paths) {
      p(sprintf("Resampling: %s", basename(rp)))
      out_name <- path(out_dir, path_file(rp))
      method <- if (grepl("id|mask|categorical", rp, ignore.case = TRUE)) "near" else "bilinear"
      
      cmd <- sprintf(
        paste(
          "gdalwarp -t_srs '%s' -tr %.2f %.2f -r %s",
          "-te %.2f %.2f %.2f %.2f",
          "-multi -wo NUM_THREADS=%d -co COMPRESS=DEFLATE -co TILED=YES",
          "%s %s"
        ),
        crs(template),
        res(template)[1], res(template)[2],
        method,
        xmin(template), ymin(template), xmax(template), ymax(template),
        n_threads,
        shQuote(rp), shQuote(out_name)
      )
      system(cmd, ignore.stdout = TRUE, ignore.stderr = TRUE)
      out_files <- c(out_files, out_name)
    }
  })
  
  message("✅ All rasters resampled.")
  return(out_files)
}

# Run
resampled_files <- resample_to_template(template_path, subset_files, tmp_dirs$resample, n_threads)

In [ ]:
# ============================================================
#  4. Stack rasters into final multi-year raster
# ============================================================

stack_rasters <- function(resampled_files, output_path, tmp_dir, n_threads = 8) {
  dir_create(tmp_dir)
  q <- function(x) shQuote(normalizePath(x, mustWork = FALSE))
  
  band_vrts <- c()
  band_years <- list()
  
  with_progress({
    p <- progressor(along = resampled_files)
    for (rf in resampled_files) {
      p(sprintf("Processing: %s", basename(rf)))
      r <- rast(rf)
      n_bands <- nlyr(r)
      base <- path_ext_remove(path_file(rf))
      layer_years <- as.numeric(sub(".*_(\\d{4})$", "\\1", names(r)))
      
      for (i in seq_len(n_bands)) {
        b_vrt <- path(tmp_dir, sprintf("%s_band%02d.vrt", base, i))
        system(sprintf("gdal_translate -of VRT -b %d %s %s", i, q(rf), q(b_vrt)),
               ignore.stdout = TRUE, ignore.stderr = TRUE)
        band_vrts <- c(band_vrts, b_vrt)
        band_years[[b_vrt]] <- layer_years[i]
      }
    }
  })
  
  ordered_vrts <- band_vrts[order(unlist(band_years))]
  
  vrt_file <- path(tmp_dir, "stack.vrt")
  system(paste("gdalbuildvrt -separate", q(vrt_file), paste(q(ordered_vrts), collapse = " ")))
  system(sprintf("gdal_translate %s %s -co COMPRESS=DEFLATE -co TILED=YES -co BIGTIFF=YES -co NUM_THREADS=%d",
                 q(vrt_file), q(output_path), n_threads))
  
  message("✅ Final stacked raster: ", output_path)
  return(output_path)
}

# Run
stacked_path <- stack_rasters(resampled_files, output_stack_path, tmp_dirs$vrt, n_threads)

In [ ]:
# ============================================================
#  5. Apply Forest Mask
# ============================================================

apply_forest_mask <- function(stack_path, mask_path, template_path, out_path) {
  template <- rast(template_path)
  mask_r <- rast(mask_path)
  mask_r <- project(mask_r, crs(template), method = "near")
  mask_r <- resample(mask_r, template, method = "near")
  
  stack_r <- rast(stack_path)
  masked_r <- mask(stack_r, mask_r, maskvalue = 0, updatevalue = NA)
  
  writeRaster(masked_r, out_path, overwrite = TRUE,
              gdal = c("COMPRESS=DEFLATE", "TILED=YES"))
  message("✅ Forest mask applied: ", out_path)
  out_path
}

# Run
masked_path <- apply_forest_mask(stacked_path, forest_mask_path, template_path,
                                 here("data/derived/disturbance_stack_2000_2020_masked.tif"))

In [ ]:
# ============================================================
#  6. QA / Validation Step
# ============================================================

validate_final_stack <- function(stack_path, years) {
  r <- rast(stack_path)
  
  # Check bands and names
  n_bands <- nlyr(r)
  expected_bands <- length(years)
  if (n_bands != expected_bands) {
    warning(sprintf("⚠️ Expected %d bands (2000–2020) but found %d.", expected_bands, n_bands))
  } else {
    message("✅ Band count matches expected years.")
  }
  
  # Check NA proportion
  na_frac <- global(is.na(r[[1]]), "mean", na.rm = FALSE)[[1]]
  message(sprintf("ℹ️ First band NA fraction: %.2f%%", 100 * na_frac))
  
  # Spatial checks
  message("ℹ️ CRS:", crs(r))
  message("ℹ️ Extent:", paste(signif(ext(r)), collapse = ", "))
  message("ℹ️ Resolution:", paste(res(r), collapse = " x "))
  
  # Optional quick visual sample check
  plot(r[[1]], main = "First band (2000)")
  plot(r[[n_bands]], main = "Last band (2020)")
  
  message("✅ Validation complete.")
}

# Run validation
validate_final_stack(stacked_path, years)

In [ ]:
# ============================================================
#  7. Rename bands
# ============================================================

# Load stacked raster
stacked <- rast(output_stack_path)

# Create band names
raster_names <- c("wildfire", "biotic", "pdsi", "hd")   # in same order as input
years <- 2000:2020

band_names <- c()
for (yr in years) {
  for (r in raster_names) {
    band_names <- c(band_names, paste0(r, "_", yr))
  }
}

# Assign names
names(stacked) <- band_names

# Save renamed stack
writeRaster(stacked, output_stack_path, overwrite = TRUE, datatype="FLT4S", gdal=c("COMPRESS=DEFLATE", "TILED=YES"))